In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/test-data/Test.csv
/kaggle/input/lemm-data/lemm_data.csv
/kaggle/input/book-rating-prediction/sample_submission.csv
/kaggle/input/book-rating-prediction/Train.csv
/kaggle/input/book-rating-prediction/Test.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU
import tensorflow as tf

In [3]:
data = pd.read_csv("/kaggle/input/book-rating-prediction/Train.csv")

SyntaxError: unmatched ')' (3090605755.py, line 1)

In [4]:
df = pd.DataFrame(data)
df

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating
0,327858f6d967ef1567459b84252ac71a,8909152,e8cb23191d6c27e930243a08ff826395,"Really, I meant to get Landline when I checked...",Mon Aug 24 10:09:11 -0700 2015,Mon Aug 24 18:50:11 -0700 2015,Sat Aug 22 00:00:00 -0700 2015,NaN,1,2,4
1,b1bb017c681370fddd19bab11f4eb22a,13152,953dfd48b372f081b5f82ce1def753f7,Update - 01/08/2016 They are making a Maximum ...,Sun Jun 16 01:49:16 -0700 2013,Sat Oct 22 04:08:59 -0700 2016,NaN,NaN,1,0,4
2,6fb896f0ccaeb445e2c9f580bff8f65d,9533378,48509a6f6128d4f2ca243e04a0cdc896,I feel like I've read so many Urban Fantasy bo...,Tue Nov 08 21:15:28 -0800 2016,Wed Dec 28 16:40:44 -0800 2016,Sun Nov 13 00:00:00 -0800 2016,NaN,1,0,3
3,2edbb0dcf49ad138ef79bd6b5f4ba390,12067,a09f7ff4eca0c8c2fbaacf4baf6b114f,Reread in December 2009. \n Simply a fantastic...,Thu Oct 08 08:21:11 -0700 2009,Sat Jan 02 07:32:11 -0800 2010,Sat Jan 02 00:00:00 -0800 2010,NaN,1,0,5
4,cc3ce566b0313a2f02d6ab246c990bce,20443207,93b0128f768ee9c1af8864f566e3a7b6,BIG ASS DNF \n Ughhh. I'm so mad at myself for...,Thu Jan 28 02:16:44 -0800 2016,Thu Jan 28 02:35:30 -0800 2016,NaN,NaN,7,3,1
...,...,...,...,...,...,...,...,...,...,...,...
629995,c62f6649b458d3d337851f2ebe125a8a,15858248,175e2381b339ff61b3448b6e9d2b8ee3,Now look at that cover! My very first impressi...,Mon May 19 02:04:46 -0700 2014,Fri Jun 30 17:27:16 -0700 2017,Wed Jan 21 00:00:00 -0800 2015,Thu Jan 15 00:00:00 -0800 2015,29,7,5
629996,288dc8c9871098c8a1b680db829275b4,13188615,124090e14f4ad3ae1f72a50d00f08494,this book was good but I didn't like it as muc...,Tue Aug 28 04:53:49 -0700 2012,Wed Dec 03 09:43:31 -0800 2014,Wed Dec 03 09:43:31 -0800 2014,Sun Nov 30 00:00:00 -0800 2014,0,0,3
629997,dc0d067a33a72b35af9e07b1522b827e,13330943,5b07fb855a48b475f594577f8f44f1f6,4/5 stars \n I went into this book with low ex...,Mon Oct 05 17:06:59 -0700 2015,Mon Feb 20 17:59:29 -0800 2017,Thu Jan 14 09:29:23 -0800 2016,Fri Jan 08 00:00:00 -0800 2016,1,0,4
629998,945f2e70af82deb0767d739cd3b8a15b,5899779,fb23f44d931b39cb49f496d4613dbba0,I read this concurrently with the original wor...,Thu Apr 09 08:41:41 -0700 2009,Wed May 13 08:53:29 -0700 2009,Sat May 09 00:00:00 -0700 2009,NaN,0,0,2


In [5]:
df['rating'].value_counts()

rating
4    219581
5    185505
3    132280
2     50839
0     21692
1     20103
Name: count, dtype: int64

In [6]:
df['length']=df['review_text'].apply(lambda x:len(x))
l=(df['length'])
max(l)

19826

In [45]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [46]:
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)
def tokenization(text):
    tokens = re.split(r'\W+', text)
    return tokens
def stemming(tokens):
    porter_stemmer = PorterStemmer()
    return [porter_stemmer.stem(word) for word in tokens]

# # Function for lemmatization
# def lemmatization(tokens):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(word) for word in tokens]
#Removing the noisy text
def denoise_text(text):
    text = text.lower() 
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text= tokenization(text)
#     text = lemmatization(text)
    return text

df['cleaned_review'] = df['review_text'].apply(denoise_text)
df['cleaned_review'] = df['cleaned_review'].apply(stemming)
df.head()


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/tmp/ipykernel_47/2964451503.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating,length,cleaned_review
0,327858f6d967ef1567459b84252ac71a,8909152,e8cb23191d6c27e930243a08ff826395,"Really, I meant to get Landline when I checked...",Mon Aug 24 10:09:11 -0700 2015,Mon Aug 24 18:50:11 -0700 2015,Sat Aug 22 00:00:00 -0700 2015,NaN,1,2,4,1788,"[realli, meant, get, landlin, check, one, reas..."
1,b1bb017c681370fddd19bab11f4eb22a,13152,953dfd48b372f081b5f82ce1def753f7,Update - 01/08/2016 They are making a Maximum ...,Sun Jun 16 01:49:16 -0700 2013,Sat Oct 22 04:08:59 -0700 2016,NaN,NaN,1,0,4,124,"[updat, make, maximum, ride, movi, and, look, ..."
2,6fb896f0ccaeb445e2c9f580bff8f65d,9533378,48509a6f6128d4f2ca243e04a0cdc896,I feel like I've read so many Urban Fantasy bo...,Tue Nov 08 21:15:28 -0800 2016,Wed Dec 28 16:40:44 -0800 2016,Sun Nov 13 00:00:00 -0800 2016,NaN,1,0,3,503,"[feel, like, ive, read, mani, urban, fantasi, ..."
3,2edbb0dcf49ad138ef79bd6b5f4ba390,12067,a09f7ff4eca0c8c2fbaacf4baf6b114f,Reread in December 2009. \n Simply a fantastic...,Thu Oct 08 08:21:11 -0700 2009,Sat Jan 02 07:32:11 -0800 2010,Sat Jan 02 00:00:00 -0800 2010,NaN,1,0,5,191,"[reread, decemb, simpli, fantast, read, full, ..."
4,cc3ce566b0313a2f02d6ab246c990bce,20443207,93b0128f768ee9c1af8864f566e3a7b6,BIG ASS DNF \n Ughhh. I'm so mad at myself for...,Thu Jan 28 02:16:44 -0800 2016,Thu Jan 28 02:35:30 -0800 2016,NaN,NaN,7,3,1,276,"[big, ass, dnf, ughhh, im, mad, pick, even, ca..."


In [47]:
df['cleaned_review']

0         [realli, meant, get, landlin, check, one, reas...
1         [updat, make, maximum, ride, movi, and, look, ...
2         [feel, like, ive, read, mani, urban, fantasi, ...
3         [reread, decemb, simpli, fantast, read, full, ...
4         [big, ass, dnf, ughhh, im, mad, pick, even, ca...
                                ...                        
629995    [look, cover, first, impress, first, saw, cove...
629996                [book, good, like, much, first, book]
629997    [, star, went, book, low, expect, night, circu...
629998    [read, concurr, origin, work, necessari, order...
629999    [shadow, georg, catlin, paint, direct, forc, p...
Name: cleaned_review, Length: 630000, dtype: object

In [49]:
# from imblearn.over_sampling import SMOTE
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import Pipeline
# from sklearn.svm import SVC
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics import classification_report, accuracy_score, f1_score
# import pandas as pd

# # Assuming df is your DataFrame with 'cleaned_review' and 'rating' columns

# # Step 1: Prepare the Data
# X = df['cleaned_review']
# y = df['rating']

# # Convert class labels to strings
# y = y.astype(str)

# # Step 2: Split the Data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Convert the lists to strings
# X_train = [" ".join(review) for review in X_train]
# X_test = [" ".join(review) for review in X_test]

# # Step 3: TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer()
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [63]:
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.over_sampling import RandomOverSampler
# unique_classes = df['rating'].unique()

# unique_classes = df['rating'].unique()
# sampling_strategy_undersample = {str(cls): 90000 for cls in unique_classes if str(cls) in ['3']}

# undersampler = RandomUnderSampler(sampling_strategy=sampling_strategy_undersample, random_state=42)
# X_resampled, y_resampled = undersampler.fit_resample(X_train_tfidf, y_train)

# # Step 2: Oversample classes '0', '1', and '2'
# oversampler = RandomOverSampler(sampling_strategy={'0': 50000, '1': 50000}, random_state=42)
# X_resampled, y_resampled = oversampler.fit_resample(X_resampled, y_resampled)


In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import classification_report, accuracy_score, f1_score
# import pandas as pd

# # Step 5: Build and Train the Model with Logistic Regression
# logreg_model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs', random_state=42)
# logreg_model.fit(X_resampled, y_resampled)

# # Step 6: Evaluate the Model
# y_pred = logreg_model.predict(X_test_tfidf)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy:", accuracy)

# # Calculate F1 score
# f1 = f1_score(y_test, y_pred, average='weighted')
# print("F1 Score:", f1)

# # Print classification report
# print("Classification Report:\n", classification_report(y_test, y_pred))


In [4]:
df = pd.read_csv("/kaggle/input/lemm-data/lemm_data.csv")

In [7]:
df

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments,rating,preprocessed_text
0,d9a0f14b836e2634b89a6a7d4d9aa184,23308084,7bde8725cbb9ceb697c72c12262dff53,My only complaint is this isn't a trilogy...I ...,Thu Jul 23 22:27:34 -0700 2015,Wed May 04 20:04:40 -0700 2016,Wed May 04 00:00:00 -0700 2016,Mon Apr 25 00:00:00 -0700 2016,1,1,5,"['complaint', 'be', 'nt', 'trilogyi', 'want', ..."
1,3af7e1cda1d80d6a6d73e06eab301368,23310751,a9aa8356ef1ca470c98712e17005517b,i read this 2 days ago.... and remember nothin...,Thu Oct 13 19:33:58 -0700 2016,Thu Oct 13 19:34:28 -0700 2016,Mon Oct 10 00:00:00 -0700 2016,NaN,3,0,2,"['read', 'day', 'ago', 'rememb', 'noth', 'disa..."
2,dd669721e136c1be47d739b14fa23d20,210252,8739baeb543858142605442041d79524,"I wasn't a fan of the first Monster Blood, and...",Thu Feb 27 01:44:54 -0800 2014,Sun Oct 18 20:46:53 -0700 2015,Thu Feb 27 00:00:00 -0800 2014,NaN,0,0,2,"['be', 'nt', 'fan', 'first', 'monster', 'blood..."
3,aec624fd1ad0034f2553b7dc55ee1cd0,472392,1791472bb94c9733802303ecf34d9c53,Love Theo so much. The interaction between she...,Sun Aug 28 08:19:18 -0700 2016,Sat Sep 03 17:42:00 -0700 2016,Fri Sep 02 03:12:59 -0700 2016,Sun Aug 28 00:00:00 -0700 2016,0,0,4,"['love', 'theo', 'much', 'interact', 'brother'..."
4,d67aef0988e1814a819259eb11c92788,17675462,d98212782db1271607a94c5836ef6189,"a good book, just had to get through the middl...",Mon Jan 11 07:20:47 -0800 2016,Mon Mar 14 07:30:27 -0700 2016,Sun Mar 13 00:00:00 -0800 2016,Mon Jan 11 00:00:00 -0800 2016,0,0,4,"['good', 'book', 'get', 'middl', 'part', 'look..."
...,...,...,...,...,...,...,...,...,...,...,...,...
629995,34d07fb9e04bfbbb17371435223b120e,296182,3de0c7af9700e937b488e129ca3c9cc9,**edit 11/26/13 \n My advice regarding the Vor...,Sat Jul 06 23:27:10 -0700 2013,Thu Jul 10 23:35:49 -0700 2014,NaN,NaN,8,3,4,"['edit', 'advic', 'regard', 'vorkosigan', 'sag..."
629996,c3d7ca2c133140d684ab4d958d5e4ee9,5776788,96a1196bfce7fc2d4d4e589d990badc9,This is not a book I would likely have picked ...,Tue Oct 19 17:38:21 -0700 2010,Wed Dec 22 09:46:33 -0800 2010,Tue Dec 21 00:00:00 -0800 2010,Sat Dec 18 00:00:00 -0800 2010,0,0,3,"['book', 'would', 'like', 'pick', 'read', 'get..."
629997,60982541be85a0611e9634b4f63d0cb0,3648,a857718ca7e70b8c0ffc5ead14512fb8,This book's summary boasts a spectacular story...,Thu Jun 08 22:25:07 -0700 2017,Thu Jun 08 22:28:54 -0700 2017,Fri Jun 09 07:16:37 -0700 2017,NaN,13,0,2,"['book', 'summari', 'boast', 'spectacular', 's..."
629998,b43eaf1760e1b11bc224815a3f3c48a3,13507212,393045562fb081cf0a6975a2f6b91908,Excellent sequel to Wolf Hall- covering a shor...,Sun Jan 08 08:38:38 -0800 2017,Sun Jan 15 06:16:15 -0800 2017,NaN,NaN,0,0,5,"['excel', 'sequel', 'wolf', 'hall', 'cover', '..."


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, f1_score
import pandas as pd

# Assuming df is your DataFrame with 'cleaned_review' and 'rating' columns

# Step 1: Prepare the Data
X = df['preprocessed_text']
y = df['rating']

# Step 2: Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Step 3: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Step 4: Build and Train the Model
logreg_model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='sag', random_state=42)
logreg_model.fit(X_train_tfidf, y_train)

# Step 5: Evaluate the Model
y_pred = logreg_model.predict(X_test_tfidf)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", f1)

# Print classification report
print("Classification Report:\n", classification_report(y_test, y_pred))




Accuracy: 0.5192539682539683
F1 Score: 0.5120267474497875
Classification Report:
               precision    recall  f1-score   support

           0       0.52      0.25      0.34      4280
           1       0.47      0.28      0.35      3997
           2       0.42      0.27      0.33     10289
           3       0.45      0.41      0.43     26306
           4       0.49      0.60      0.54     43846
           5       0.62      0.62      0.62     37282

    accuracy                           0.52    126000
   macro avg       0.50      0.41      0.44    126000
weighted avg       0.52      0.52      0.51    126000



In [ ]:
df[["rating"]]

In [ ]:
data1 = pd.read_csv("/kaggle/input/book-rating-prediction/Test.csv")

In [ ]:
df1 = pd.DataFrame(data1)
df1

In [ ]:
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)
def tokenization(text):
    tokens = re.split(r'\W+', text)
    return tokens
def stemming(tokens):
    porter_stemmer = PorterStemmer()
    return [porter_stemmer.stem(word) for word in tokens]

# # Function for lemmatization
# def lemmatization(tokens):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(word) for word in tokens]
#Removing the noisy text
def denoise_text(text):
    text = text.lower() 
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text= tokenization(text)
#     text = lemmatization(text)
    return text

df1['cleaned_review'] = df1['review_text'].apply(denoise_text)
# df1['cleaned_review'] = df1['cleaned_review'].apply(stemming)



In [ ]:
df1.head()

In [ ]:

X_test_new = df1['cleaned_review']


X_test_new = [" ".join(review) for review in X_test_new]


X_test_tfidf_new = tfidf_vectorizer.transform(X_test_new)


y_pred_new = logreg_model.predict(X_test_tfidf_new)


df1['rating'] = y_pred_new


print(df1[['cleaned_review', 'rating']])



In [43]:
submission =df1[["review_id","rating"]]
submission


,review_id,rating
0,606d3b7ba5cc90e4069b1e225b84deea,5
1,70388d316638176b827ea060839971ef,4
2,840a4fb63336a3fc2213e5016cafbca2,3
3,6c5865dc54856d1dcb317c3dd42215a8,5
4,0fdf6c933c0b299ebf5053b8d9a2f950,4
...,...,...
269995,f69f612c8a460fc5d13ea414d370f636,4
269996,78e5e4d5427adab67653bcb5331f1da0,5
269997,1b7dfa5240b9b9412567c98f9c592eae,5
269998,587ca6fc22ce2ca7b7cffca98e6a4b4e,0


In [ ]:
submission=submission.to_csv("submission07.csv",index=None)
pd.read_csv("submission07.csv")